In [1]:
mport os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.ETM import main
import torch
import numpy as np

In [2]:
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

True

In [3]:
# Load model
model = main.ETM_Wrapper()

In [4]:
model.set_model(dataset, 10, 800, 300, 300, 'relu', None, True, 0, 0.005, 'adam',
                32, 2)

In [8]:
y, z, t = model.train_model()

1268